# Fetch Reference Data from Scopus API

Resouces:

- [Scopus Abstract Retrieval Views](https://dev.elsevier.com/sc_abstract_retrieval_views.html)
- [Scopus Retrieval API](https://dev.elsevier.com/documentation/AbstractRetrievalAPI.wadl)
- [Interactive Scopus API](https://dev.elsevier.com/scopus.html)
- [API Settings (rate limits)](https://dev.elsevier.com/api_key_settings.html)
- Remember Logging In to Cisco VPN!!!


In this notebook, we use the `ScopusReferenceFetech` (from src.data_fetching.ScopusReferenceFetcher import ScopusReferenceFetcher). The class was created to send requests to the scopus search api and retrieve the references for a given scopus id.

In this class, we are applying the functionility to a list of scopus ids.

1. We provide the full dataframe with the scopus ids - the code runs until the api rate limit of 10,000 requests per week is reached.
2. It saves after every 500 requests.
3. After the first run, and the first rate limit is reached, we use addittional api keys to continue the process.
4. To do that, we first load all previously saved files from the data folder.
5. We save the 'eid's of the files that were already processed in a list to filter the dataframe to those that are not yet processed.
6. We retrieve the highest number of batches to know where to continue from.


In [3]:
import pandas as pd
import json
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from tqdm import tqdm
import json
import logging
from math import ceil
import sys
import logging
import datetime
import os
import os
import json
import pandas as pd
import logging
import datetime
import json
from tqdm import tqdm
from math import ceil
import sys
%load_ext autoreload
%autoreload 2


from src.data_fetching.ScopusReferenceFetcher import ScopusReferenceFetcher
from src.data_fetching.ScopusReferenceFetcherUtils import (
    ScopusRefFetcherPrep,
    ScopusRefFetcherProcessor,
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Initial Reference Fetching


## get api key and load data


In [7]:
api_keys = ScopusRefFetcherPrep.get_api_keys()
api_key = api_keys["api_key_A"]

# load data
df_path = "../data/02-clean/articles/scopus_cleaned_20250326_081230.pkl"
df = pd.read_pickle(df_path)

Comment:  rate limits are 40,000 per week for api_key_A and 10,000 for every other key


## setup logging and process the data


In [ ]:
log_directory = "../data/02-clean/references/"
ScopusRefFetcherProcessor.setup_logging(log_directory, log_level=logging.INFO)

# max batch is 0 since no data has been fetched yet
max_batch = 0
# Process batches
processor.process_scopus_batches(
    api_key=api_key,
    df_to_fetch=df,
    data_path="../data/01-raw/scopus/references/",
    last_processed_batch=max_batch,
    batch_size=500,
)

# Subsequent Reference Fetching


# prep


In [4]:
prepper = ScopusRefFetcherPrep()
api_keys = prepper.get_api_keys()
api_key = api_keys["api_key_deb"]

refs_path = "../data/01-raw/scopus/references/"
eids, max_batch = prepper.load_fetched_reference_data(refs_path)

df_path = "../data/01-raw/scopus/articles/final_scopus_results.csv"
df_filtered = prepper.load_and_filter_articles(df_path, eids)

Found 78 files with batch numbers up to 77.
Number of articles to fetch: 3744


# process


In [8]:
processor = ScopusRefFetcherProcessor()

# Set up logging
log_directory = "../notebooks/logs/scopus_fetcher"
processor.setup_logging(log_directory, log_level=logging.INFO)

# Process batches
processor.process_scopus_batches(
    api_key=api_key,
    df_to_fetch=df_filtered,
    data_path="../data/01-raw/scopus/references/",
    last_processed_batch=max_batch,
    batch_size=500,
)

100%|██████████| 8/8 [1:41:40<00:00, 762.51s/it]
